In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('train.csv')
df.info()
df.isna().sum()
df.duplicated().sum()
df.drop_duplicates()
label =  LabelEncoder()
df['model'] = label.fit_transform(df['model'])
df['motor_type'] = label.fit_transform(df['motor_type'])
df['wheel'] = label.fit_transform(df['wheel'])
df['color'] = label.fit_transform(df['color'])
df['status'] = label.fit_transform(df['status'])
df['type'] = label.fit_transform(df['type'])
def convert_running(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 
df['running'] = df['running'].apply(convert_running)
df['running'] = df['running'].str.replace('miles', '')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   int32  
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   int32  
 3   running       1642 non-null   int64  
 4   wheel         1642 non-null   int32  
 5   color         1642 non-null   int32  
 6   type          1642 non-null   int32  
 7   status        1642 non-null   int32  
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int32(6), int64(3)
memory usage: 89.9 KB


In [15]:
df['running'] = df['running'].str.strip()
df['running'] = pd.to_numeric(df['running'], errors='coerce')
df['running'] = df['running'].fillna(0).astype('int64')

In [19]:
df.corr()['price']

model           0.173485
year            0.638699
motor_type      0.238964
running        -0.513134
wheel                NaN
color          -0.036846
type            0.037789
status         -0.282471
motor_volume    0.000885
price           1.000000
Name: price, dtype: float64

In [21]:
x_train = df.drop(['price', 'wheel'], axis = 1)
y_train = df['price']
x_train.shape, y_train.shape

((1642, 8), (1642,))

In [23]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standardise = scalar.fit_transform(x_train)
feature_standardise

array([[ 1.52639053,  1.09243073,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       [ 0.05769246, -0.12234706,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       [-0.67665657,  0.48504183,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ],
       ...,
       [ 0.79204149,  0.48504183,  0.26699337, ...,  1.08693988,
         0.97583205, -0.1384163 ],
       [ 0.79204149,  0.63688906,  0.26699337, ...,  1.08693988,
        -0.45353122, -0.1384163 ],
       [ 1.52639053,  1.09243073,  0.26699337, ..., -0.03413756,
        -0.45353122, -0.1384163 ]])

In [31]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=7)  

knn.fit(feature_standardise, y_train)

KNeighborsRegressor(n_neighbors=7)

In [29]:
from sklearn.model_selection import GridSearchCV


param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11]}


grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(feature_standardise, y_train)

print(f'Best number of neighbors: {grid_search.best_params_["n_neighbors"]}')

Best number of neighbors: 7


In [49]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = knn.predict(feature_standardise)
mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
mse = mean_absolute_error(y_train, y_pred)
print(f'Mean Absolute Error: {mse:.2f}')
r2 = r2_score(y_train, y_pred)
print(f"r2 Score = {r2}")

Mean Squared Error: 14962522.68
Mean Absolute Error: 2246.23
r2 Score = 0.709267028156181


In [104]:
test = pd.read_csv('test.csv')
test

,Id,model,year,motor_type,running,wheel,color,type,status,motor_volume
0,0,kia,2020,petrol,24000 km,left,black,sedan,excellent,2.0
1,1,nissan,2017,petrol,85000 miles,left,white,suv,excellent,2.0
2,2,hyundai,2021,petrol,30000 miles,left,white,sedan,excellent,2.0
3,3,kia,2018,petrol,53000 miles,left,silver,sedan,excellent,2.0
4,4,mercedes-benz,2003,petrol,230000 km,left,black,Universal,normal,1.8
...,...,...,...,...,...,...,...,...,...,...
406,406,nissan,2021,petrol,33400 km,left,black,suv,excellent,2.0
407,407,hyundai,2017,petrol,60000 miles,left,other,sedan,excellent,2.0
408,408,mercedes-benz,2012,petrol,218000 km,left,white,sedan,good,2.0
409,409,kia,2020,petrol,40000 miles,left,red,sedan,good,2.0


In [106]:
from sklearn.preprocessing import LabelEncoder
label =  LabelEncoder()
test['model'] = label.fit_transform(test['model'])
test['motor_type'] = label.fit_transform(test['motor_type'])
test['wheel'] = label.fit_transform(test['wheel'])
test['color'] = label.fit_transform(test['color'])
test['status'] = label.fit_transform(test['status'])
test['type'] = label.fit_transform(test['type'])
def convert_runnings(value):
    if 'km' in value:
        kilometers = int(value.split()[0])  
        miles = kilometers * 0.621371        
        return f"{miles:.2f} miles"          
    elif 'miles' in value:
        return value  
    else:
        return value 
test['running'] = test['running'].apply(convert_runnings)
test['running'] = test['running'].str.replace('miles', '')

In [108]:
test.head()

,Id,model,year,motor_type,running,wheel,color,type,status,motor_volume
0,0,1,2020,1,14912.90,0,1,3,1,2.0
1,1,3,2017,1,85000,0,13,4,1,2.0
2,2,0,2021,1,30000,0,13,3,1,2.0
3,3,1,2018,1,53000,0,11,3,1,2.0
4,4,2,2003,1,142915.33,0,1,1,4,1.8


In [118]:
test

,Id,model,year,motor_type,running,wheel,color,type,status,motor_volume
0,0,1,2020,1,14912.90,0,1,3,1,2.0
1,1,3,2017,1,85000,0,13,4,1,2.0
2,2,0,2021,1,30000,0,13,3,1,2.0
3,3,1,2018,1,53000,0,11,3,1,2.0
4,4,2,2003,1,142915.33,0,1,1,4,1.8
...,...,...,...,...,...,...,...,...,...,...
406,406,3,2021,1,20753.79,0,1,4,1,2.0
407,407,0,2017,1,60000,0,9,3,1,2.0
408,408,2,2012,1,135458.88,0,13,3,2,2.0
409,409,1,2020,1,40000,0,10,3,2,2.0


In [110]:
x_test = test

In [112]:
x_test = x_test.drop(['Id', 'wheel'], axis = 1)

In [114]:
from sklearn.preprocessing import StandardScaler
scalar =  StandardScaler()
feature_standard = scalar.fit_transform(x_test)
feature_standard

array([[-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       [ 0.8692927 ,  0.31182288,  0.1137996 , ...,  1.45476435,
        -0.4470452 , -0.10974538],
       [-1.36836471,  0.9242922 ,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538],
       ...,
       [ 0.1234069 , -0.45376377,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-0.62247891,  0.77117487,  0.1137996 , ..., -0.05892716,
         1.04673998, -0.10974538],
       [-1.36836471,  0.31182288,  0.1137996 , ..., -0.05892716,
        -0.4470452 , -0.10974538]])

In [116]:
y_predict = knn.predict(feature_standard)
y_predict
df_train = pd.DataFrame(y_predict, columns=['price'])
df_train

,price
0,19785.714286
1,16150.000000
2,24578.571429
3,16585.714286
4,7000.000000
...,...
406,22900.000000
407,13592.857143
408,11614.285714
409,14014.285714


In [120]:
combined_df = pd.concat([test['Id'], df_train], axis=1)

combined_df

,Id,price
0,0,19785.714286
1,1,16150.000000
2,2,24578.571429
3,3,16585.714286
4,4,7000.000000
...,...,...
406,406,22900.000000
407,407,13592.857143
408,408,11614.285714
409,409,14014.285714


In [124]:
combined_df.to_csv('Fi_sub.csv', index=False)

In [128]:
sub = pd.read_csv('Fi_sub.csv')
sub

,Id,price
0,0,19785.714286
1,1,16150.000000
2,2,24578.571429
3,3,16585.714286
4,4,7000.000000
...,...,...
406,406,22900.000000
407,407,13592.857143
408,408,11614.285714
409,409,14014.285714


In [132]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
reg = RandomForestRegressor(n_estimators=100)
model = RandomForestRegressor()
model.fit(feature_standardise,y_train)

RandomForestRegressor()

In [136]:
mse_R = mean_squared_error(y_train,  model.predict(feature_standardise))
print(f"Mean Squared Error = {mse_R}")

mae_R = mean_squared_error(y_train,  model.predict(feature_standardise))
print(f"Mean Absolute Error = {mae_R}")

Mean Squared Error = 2105810.163662195
Mean Absolute Error = 2105810.163662195


In [138]:
r2_R = r2_score(y_train,  model.predict(feature_standardise))
print(f"r2 Score = {r2_R}")

r2 Score = 0.9590825384054309


In [140]:
y_pred = model.predict(feature_standard)
y_pred
df_tra = pd.DataFrame(y_pred, columns=['price'])
df_tra

,price
0,16461.50
1,16194.21
2,25676.00
3,14777.75
4,5139.00
...,...
406,30478.07
407,15263.50
408,15154.50
409,16704.69


In [142]:
comb_df = pd.concat([test['Id'], df_tra], axis=1)

comb_df

,Id,price
0,0,16461.50
1,1,16194.21
2,2,25676.00
3,3,14777.75
4,4,5139.00
...,...,...
406,406,30478.07
407,407,15263.50
408,408,15154.50
409,409,16704.69


In [144]:
comb_df.to_csv('New_car.csv', index=False)
su = pd.read_csv('New_car.csv')
su

,Id,price
0,0,16461.50
1,1,16194.21
2,2,25676.00
3,3,14777.75
4,4,5139.00
...,...,...
406,406,30478.07
407,407,15263.50
408,408,15154.50
409,409,16704.69
